In [49]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import h5py 
import cv2 
from numba import cuda

### About the Dataset:

This brain tumor dataset containing 3064 T1-weighted contrast-inhanced images
from 233 patients with three kinds of brain tumor: meningioma (708 slices),
glioma (1426 slices), and pituitary tumor (930 slices). Due to the file size
limit of repository, we split the whole dataset into 4 subsets, and achive
them in 4 .zip files with each .zip file containing 766 slices.The 5-fold
cross-validation indices are also provided.

---

This data is organized in matlab data format (.mat file). Each file stores a struct
containing the following fields for an image:

cjdata.label: 1 for meningioma, 2 for glioma, 3 for pituitary tumor
cjdata.PID: patient ID
cjdata.image: image data
cjdata.tumorBorder: a vector storing the coordinates of discrete points on tumor border.
For example, [x1, y1, x2, y2,...] in which x1, y1 are planar coordinates on tumor border.
It was generated by manually delineating the tumor border. So we can use it to generate
binary image of tumor mask.
cjdata.tumorMask: a binary image with 1s indicating tumor region

---

This data was used in the following paper:

1. Cheng, Jun, et al. "Enhanced Performance of Brain Tumor Classification via Tumor Region Augmentation
   and Partition." PloS one 10.10 (2015).
2. Cheng, Jun, et al. "Retrieval of Brain Tumors by Adaptive Spatial Pooling and Fisher Vector
   Representation." PloS one 11.6 (2016). Matlab source codes are available on
   [github](https://github.com/chengjun583/brainTumorRetrieval)

### Loading Data

In [39]:
# Converting all the data into Python list

data = []
for i in range(1, 3064 + 1):
    filename = str(i) + ".mat"
    tumordata = h5py.File(os.path.join('./images/', filename), "r")
    data.append(tumordata)
    if i % 500 == 0:
        print(filename)

500.mat
1000.mat
1500.mat
2000.mat
2500.mat
3000.mat


In [41]:
# Extracting image data from Python List
x_train = []
x_test = []
y_train = []
y_test = []

N = len(data)

temp = round(4 * N / 5)  # using 4/5th as training data

for i in range(temp):
    image = data[i]["cjdata"]["image"][()]

    if image.shape == (512, 512):
        image = np.expand_dims(image, axis=0)
        x_train.append(image)

        label = int(data[i]["cjdata"]["label"][()]) - 1
        y_train.append(label)

# x_test and y_test
for i in range(temp, N):
    image = np.asarray(data[i]["cjdata"]["image"][()])

    if image.shape == (512, 512):
        image = np.expand_dims(image, axis=0)
        x_test.append(image)

        label = int(data[i]["cjdata"]["label"][()]) - 1
        y_test.append(label)

Convert data to Numpy array

In [48]:
x_train = np.array(x_train).reshape(-1, 512, 512, 1)
x_test = np.array(x_test).reshape(-1, 512, 512, 1)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [47]:
tf.keras.backend.clear_session()
device = cuda.get_current_device()
device.reset()

### Model Creation using Keras Functional API

In [50]:
INPUT = keras.Input(shape=(512, 512, 1), name="image")

x1 = Conv2D(64, (22, 22), strides=2)(INPUT)
x1 = MaxPooling2D((4, 4))(x1)
x1 = BatchNormalization()(x1)

x2 = Conv2D(128, (11, 11), strides=2, padding="same")(x1)
x2 = MaxPooling2D((2, 2))(x2)
x2 = BatchNormalization()(x2)

x3 = Conv2D(256, (7, 7), strides=2, padding="same")(x2)
x3 = MaxPooling2D((2, 2))(x3)
x3 = BatchNormalization()(x3)

x4 = Flatten()(x3)
x4 = Activation("relu")(x4)

x5 = Dense(1024, "relu")(x4)

x6 = Dense(256, "relu")(x5)

x9 = Dense(3)(x6)
pred = Activation("softmax")(x9)

model = keras.Model(inputs=INPUT, outputs=pred)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 512, 512, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 246, 246, 64)      31040     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 61, 61, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 61, 61, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 128)       991360    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 128)      0     

In [51]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/20
77/77 [==============================] - 99s 1s/step - loss: 1.8329 - accuracy: 0.6051 - val_loss: 0.0751 - val_accuracy: 0.9918
Epoch 2/20
77/77 [==============================] - 86s 1s/step - loss: 0.6939 - accuracy: 0.7151 - val_loss: 12.1888 - val_accuracy: 0.0082
Epoch 3/20
77/77 [==============================] - 86s 1s/step - loss: 0.5672 - accuracy: 0.7640 - val_loss: 0.9664 - val_accuracy: 0.8369
Epoch 4/20
77/77 [==============================] - 86s 1s/step - loss: 0.4682 - accuracy: 0.8067 - val_loss: 2.1939 - val_accuracy: 0.6493
Epoch 5/20
77/77 [==============================] - 86s 1s/step - loss: 0.3735 - accuracy: 0.8391 - val_loss: 13.1451 - val_accuracy: 0.0049
Epoch 6/20
77/77 [==============================] - 87s 1s/step - loss: 0.3418 - accuracy: 0.8600 - val_loss: 8.2745 - val_accuracy: 0.1060
Epoch 7/20
77/77 [==============================] - 87s 1s/step - loss: 0.2783 - accuracy: 0.8867 - val_loss: 15.2113 - val_accuracy: 0.1387
Epoch 8/20
77/77 

In [52]:
model.save('./model/model.h5')